In [14]:
# import re

# def clean_prediction(pred_text: str):
#     if pred_text is None:
#         return ""

#     # lấy phần sau "assistant"
#     if "assistant" in pred_text:
#         pred_text = pred_text.split("assistant", 1)[-1]

#     # strip whitespace
#     return pred_text.strip()

# def extract_number(text):
#     nums = re.findall(r"-?\d+\.?\d*", text)
#     if len(nums) == 0:
#         return None
#     try:
#         return float(nums[0])
#     except:
#         return None

# def get_answer_type(gt: str):
#     gt = gt.strip().lower()

#     if gt in ["yes", "no"]:
#         return "boolean"

#     try:
#         float(gt)
#         return "numeric"
#     except:
#         return "string"

# def evaluate_sample(pred_text, gt):
#     pred_text = clean_prediction(pred_text)
#     answer_type = get_answer_type(gt)

#     # YES / NO
#     if answer_type == "boolean":
#         pred = pred_text.lower()
#         if "yes" in pred:
#             return gt.lower() == "yes"
#         if "no" in pred:
#             return gt.lower() == "no"
#         return False

#     # NUMERIC
#     if answer_type == "numeric":
#         pred_num = extract_number(pred_text)
#         gt_num = float(gt)

#         if pred_num is None:
#             return False

#         # exact match
#         if pred_num == gt_num:
#             return True

#         # relaxed (5%)
#         if abs(pred_num - gt_num) / max(1.0, abs(gt_num)) <= 0.05:
#             return True

#         return False

#     # STRING
#     pred = pred_text.lower()
#     gt = gt.lower()

#     return gt in pred

In [15]:
# import json

# def load_jsonl(path):
#     records = []
#     with open(path, "r", encoding="utf-8") as f:
#         for line in f:
#             line = line.strip()
#             if not line:
#                 continue
#             records.append(json.loads(line))
#     return records

In [16]:
# def evaluate_record(record):
#     pred_text = record["pred_text"]
#     gts = record["gts"]  # list

#     for gt in gts:
#         if evaluate_sample(pred_text, gt):
#             return True
#     return False
# def benchmark_jsonl(path):
#     records = load_jsonl(path)

#     total = 0
#     correct = 0

#     # optional: breakdown theo loại
#     stats = {
#         "boolean": {"correct": 0, "total": 0},
#         "numeric": {"correct": 0, "total": 0},
#         "string": {"correct": 0, "total": 0},
#     }

#     for rec in records:
#         total += 1
#         is_correct = evaluate_record(rec)
#         if is_correct:
#             correct += 1

#         # thống kê theo loại answer (dựa trên GT đầu tiên)
#         gt0 = rec["gts"][0]
#         ans_type = get_answer_type(gt0)
#         stats[ans_type]["total"] += 1
#         if is_correct:
#             stats[ans_type]["correct"] += 1

#     acc = correct / total if total > 0 else 0.0
#     return acc, stats

In [17]:
# acc, stats = benchmark_jsonl("D:\\AI\\KLTN\\chartqa_qwenvl_test_predictions.jsonl")

# print(f"Overall Accuracy: {acc:.4f}")

# for k, v in stats.items():
#     if v["total"] > 0:
#         print(f"{k}: {v['correct']}/{v['total']} = {v['correct']/v['total']:.4f}")


In [30]:

import pandas as pd
import numpy as np
import itertools
import json, csv, os, sys
from scipy.optimize import linear_sum_assignment
from tqdm import tqdm
import json

def load_jsonl(path):
    records = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            records.append(json.loads(line))
    return records

def distance(x1, x2):
    return min(1, abs((x1 - x2) / (x1+1e-15)))


def compute_cost_matrix(a1, a2):
    cost_matrix = np.zeros((len(a1), len(a2)))
    for index1, elt1 in enumerate(a1):
        for index2, elt2 in enumerate(a2):
            cost_matrix[index1, index2] = distance(elt1, elt2)
    return cost_matrix


def compute_score(lst1, lst2):
    cost_matrix = compute_cost_matrix(lst1, lst2)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    cost = cost_matrix[row_ind, col_ind].sum()
    score = 1 - cost / max(len(lst1), len(lst2))
    return score

def remove_strings(lst):
    new_lst = []
    for elt in lst:
        elt = str(elt).replace("%", '')
        # Filter out strings.
        try:
            new_lst.append(float(elt))
        except:
            continue
    return new_lst

def benchmark(path):
    records = load_jsonl(path)

    scores = []

    for record in records:
        # Ground truth numbers (list)
        gts = record["gts"]
        flattened = remove_strings(gts)

        # Prediction numbers (list or single)
        pred_nums = record["pred_num"]

        if not isinstance(pred_nums, list):
            pred_nums = [pred_nums]

        flattened2 = remove_strings(pred_nums)

        # If no numeric prediction → fail
        if len(flattened2) == 0 or len(flattened) == 0:
            score = 0.0
        else:
            score = compute_score(flattened, flattened2)

        scores.append(score)

    accuracy = sum(scores) / len(scores) if scores else 0.0
    # print("Accuracy:", accuracy)
    return accuracy



In [ ]:
scores = benchmark("D:\\AI\\KLTN\\chartqa_qwenvl_test_predictions.jsonl")
print("Final Accuracy:", scores)

Final Accuracy: 0.2012687740145013


## origin evaluation of chartQA-PRO

In [ ]:
# import ast
# import re
# from typing import List, Optional, Any, Tuple, Dict
# import pandas as pd
# from anls import anls_score
# import json
# import argparse

# def load_predictions(file_path):
#     predictions = []
#     with open(file_path, 'r', encoding='utf-8') as f:
#         for line in f:
#             line = line.strip()
#             if line:
#                 predictions.append(json.loads(line))
#     return predictions


# def fix_list_format(item: str) -> Any:
#     """
#     Standardize string representations of lists, adding quotes around elements if missing,
#     and safely evaluate to Python list. Returns original item if parsing fails.
#     """
#     if not isinstance(item, str):
#         return item
#     match = re.match(r"^\[(.*)\]$", item.strip())
#     if not match:
#         return item
#     content = match.group(1)
#     corrected = re.sub(r"(?<!['\w])(\w[^,]*?)(?!['\w])", r"'\1'", content)
#     try:
#         return ast.literal_eval(f"[{corrected}]")
#     except (SyntaxError, ValueError):
#         return item


# def parse_to_list(text: str) -> Optional[List[str]]:
#     """
#     Parses text to a list of strings if possible; strips quotes and whitespace.
#     """
#     if not isinstance(text, str):
#         return None
#     try:
#         parsed = ast.literal_eval(text)
#     except Exception:
#         return None
#     if isinstance(parsed, list):
#         return [str(x).strip(" '") for x in parsed]
#     return None


# def to_float(text: str) -> Optional[float]:
#     """
#     Converts text to float, stripping percent signs. Returns None on failure.
#     """
#     try:
#         return float(text.strip().strip('%'))
#     except ValueError:
#         return None


# def evaluate_single_answer(
#     target: str,
#     prediction: str,
#     max_relative_change: float = 0.05
# ) -> float:
#     """
#     Evaluates a single target-prediction pair:
#     - Numeric within tolerance or exact year match inside this helper.
#     - Falls back to ANLS for text.
#     """
#     t = target.strip().strip('%').strip()
#     p = prediction.strip().strip('%').strip()
#     #print("Stripped", t, p)
#     # Attempt numeric
#     t_f = to_float(t)
#     p_f = to_float(p)
#     if t_f is not None and p_f is not None:
#         if t_f == 0.0:
#             return 1.0 if p_f == 0.0 else 0.0
#         change = abs(p_f - t_f) / abs(t_f)
#         return 1.0 if change <= max_relative_change else 0.0
#     # Fallback text
#     #print("P:", p, "T: ", t)
#     return anls_score(prediction=p.lower(), gold_labels=[t.lower()], threshold=0.5)


# def relaxed_correctness_chartqapro(
#     target: str,
#     prediction: str,
#     max_relative_change: float = 0.05,
#     year_flags: Optional[List[bool]] = None,
#     always_use_exact_match: bool = False,
# ) -> float:
#     """
#     Calculates relaxed correctness between target and prediction.
#     Supports list inputs; uses year_flags to override year handling.
#     """
#     fixed_t = fix_list_format(target)
#     t_list = parse_to_list(str(fixed_t)) or [str(target)]
#     p_list = parse_to_list(str(prediction)) or [str(prediction)]
#     n = len(t_list)
#     # Expand year_flags for questions with multiple answers. 
#     if year_flags is not None and len(year_flags) < n:
#         year_flags = year_flags * n

#     # Evaluate elements
#     scores: List[float] = []
#     # print(t_list, p_list)
#     for idx in range(max(len(t_list), len(p_list))):
#         if idx >= len(t_list) or idx >= len(p_list):
#             # Model predicted more or less elements that necessary. 
#             scores.append(0.0)
#             continue
#         t_item, p_item, flag = t_list[idx], p_list[idx], year_flags[idx]
#         flag_cond = True if flag.upper()=='YES' else False
#         if flag_cond or always_use_exact_match:
#             # Exact integer match for years, fact checking, or multichoice
#             try:
#                 scores.append(1.0 if t_item.strip().lower() == p_item.strip().lower() else 0.0)
#             except ValueError:
#                 scores.append(0.0)
#         else:
#             scores.append(
#                 evaluate_single_answer(t_item, p_item, max_relative_change)
#             )
#     return sum(scores) / len(scores) if scores else 0.0


# def evaluate_predictions_chartqapro(predictions, pred_key='prediction'):
#   gts = [x['Answer'][-1].strip(".").strip("\n") for x in predictions]
#   preds = [x[pred_key].strip(".").strip("\n") for x in predictions]
#   splits = [x['Question Type'] for x in predictions]
#   year_flags =  [x['Year'] for x in predictions]
#   # Calculate accuracy by splits
#   match_nums_per_split = {}
#   match_nums = []
#   for gt, pred, split, year_flags_per_row in zip(gts, preds, splits, year_flags):
#     # check split and calculate
#     if split == 'Conversational':
#       year_flags_per_row = year_flags_per_row[-1:]
#     if split not in match_nums_per_split:
#       match_nums_per_split[split] = []

#     always_use_exact_match = True if split in ['Fact Checking', 'Multi Choice'] else False
#     score = relaxed_correctness_chartqapro(gt, pred, year_flags=year_flags_per_row)
#     #print(gt, pred, year_flags_per_row, score)
#     match_nums_per_split[split].append(score)
#     match_nums.append(score)

#   final_numbers = {}
#   for split in match_nums_per_split:
#     final_numbers[split] = sum(match_nums_per_split[split]) / len(match_nums_per_split[split])
#   final_numbers['Overall'] = sum(match_nums) / len(match_nums)
#   return final_numbers



#     # parser = argparse.ArgumentParser(description="Evaluate ChartQAPro predictions.")
#     # parser.add_argument(
#     #     "--predictions-file",
#     #     type=str,
#     #     required=True,
#     #     help="Path to the JSON file containing model predictions."
#     # )
#     # args = parser.parse_args()
#     # predictions = load_predictions(args.predictions_file)
#     # scores = evaluate_predictions_chartqapro(predictions)
#     # print("📊 Evaluation Results:")
#     # for k, v in scores.items():
#     #     print(f"  • {k:<15}: {v * 100:.2f}%")


In [ ]:
# predictions = load_predictions("D:\\AI\\KLTN\\chartqa_qwenvl_test_predictions.jsonl")
# scores = evaluate_predictions_chartqapro(predictions)
# print("📊 Evaluation Results:")
# for k, v in scores.items():
#     print(f"  • {k:<15}: {v * 100:.2f}%")